In [1]:
import json
import os
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
from baukit import Trace
import torch


In [44]:
# import my modules
import importlib
# join the path to the modules to the current working directory

import utils
importlib.reload(utils)
from utils import *

In [6]:
# download json file and save to file
# using data from 'How to catch a liar' https://arxiv.org/abs/2309.15840
if not os.path.exists('questions_1000_all.json'):
    os.system('wget https://raw.githubusercontent.com/LoryPack/LLM-LieDetector/main/data/raw_questions/questions_1000_all.json')

# load json file
with open('questions_1000_all.json') as json_file:
    data = json.load(json_file)

# convert json to dataframe
data_frame = pd.DataFrame(data).T

In [45]:
data_frame = data_frame[:200]

In [16]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"device: {device}")

device: cpu


In [33]:
# load model
model_name = "gpt2-xl"
# load tokenizer
model = AutoModelForCausalLM.from_pretrained(model_name).to(device).eval()
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.padding_side = "left"
tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
lie_format = "A liar would say: {}"
truth_format = "A truth-teller would say: {}"

In [40]:
print(check_statements(model, tokenizer, data_frame))

  0%|          | 0/102 [00:00<?, ?it/s]/home/atotheb/Documents/work/PIBBSS/meta_info_processing/utils.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if batch_gt[i].lower() in answer.lower():
100%|██████████| 102/102 [03:48<00:00,  2.24s/it]

0.9990118577075099


In [46]:
print(check_statements(model, tokenizer, data_frame, format="The following statement is False: {}"))

  0%|          | 0/20 [00:00<?, ?it/s]/home/atotheb/Documents/work/PIBBSS/meta_info_processing/utils.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if batch_gt[i].lower() in answer.lower():
100%|██████████| 20/20 [00:53<00:00,  2.69s/it]

0.78


In [48]:
print(check_statements(model, tokenizer, data_frame, format="Lie about a world fact! Lie: {}"))

100%|██████████| 20/20 [00:56<00:00,  2.83s/it]

0.725


In [49]:
print(check_statements(model, tokenizer, data_frame, format="A liar would claim the following: {}"))

100%|██████████| 20/20 [00:53<00:00,  2.68s/it]

0.755


In [51]:
print(check_statements(model, tokenizer, data_frame, format="A liar would claim: {}"))

100%|██████████| 20/20 [00:53<00:00,  2.67s/it]

0.725


In [52]:
print(check_statements(model, tokenizer, data_frame, format="A liar would say: {}"))

100%|██████████| 20/20 [00:54<00:00,  2.75s/it]

0.715


In [53]:
print(check_statements(model, tokenizer, data_frame, format="An honest person would say: {}"))

100%|██████████| 20/20 [00:54<00:00,  2.74s/it]

0.745


In [54]:
print(check_statements(model, tokenizer, data_frame, format="A truth-teller would say: {}"))

100%|██████████| 20/20 [00:56<00:00,  2.85s/it]

0.645


In [47]:
print(check_statements(model, tokenizer, data_frame, format="The following statement is true: {}"))

100%|██████████| 20/20 [00:52<00:00,  2.64s/it]

0.825


In [50]:
print(check_statements(model, tokenizer, data_frame, format="{}"))

100%|██████████| 20/20 [00:44<00:00,  2.23s/it]

0.995
